# 0. 课程前言
此为 <<人工智能安全>> 课程第四部分: 模型窃取攻击与防御实验部分.

**模型窃取攻击**的目标是通过一定手段窃取得到一个跟 受害者模型 功能和性能近似的窃取模型，从而避开昂贵的模型训练并从中获益。

<img src="./imgs/model steal.png" alt="替代模型窃取攻击示意图" style="height: 300px; max-width: 100%;">

上图为一个简单基础的模型窃取攻击过程，攻击者在黑盒环境下与受害者模型交互，获取模型的输入与输出，并不断调整查询样本来获取模型更多的决策边界信息。攻击者可以将此输入-输出作为数据集，对自己的模型进行模仿学习，得到一个与受害者模型相似的窃取模型。

一般来说，模型窃取攻击的主要目标包括：
1. 低代价：以远低于受害者模型训练成本的代价获得一个可免费使用的窃取模型；
2. 高收益：窃取得到的模型与受害者模型的功能和性能相当；
3. 低风险：在窃取过程中可以避开相关检测并在窃取后无法被溯源。

# 1. 模型窃取攻击
模型窃取攻击有多种方式：基于**方程式求解**的模型窃取攻击、基于**替代模型**的窃取攻击、基于**元模型**的窃取攻击。

受篇幅影响，本次实验主要将基于替代模型的窃取攻击，有兴趣的同学可自行搜索学习其他类别的攻击方法。

## 1.1 基于替代模型的窃取攻击
攻击主要思路：攻击者 $A$ 在不知道受害者模型 $f(\cdot)$ 任何先验知识情况下，向受害者模型输入查询样本 $x$ ，得到受害者模型的预测输出 $f(x)$ 。随后，攻击者根据输入和输出构建替代训练数据集 $\mathcal D ' = {(x, f(x))}^m_{i=1}$ 。

实际上，替代数据集已经完成了对袁术训练数据的（部分）提取。在替代数据集 $\mathcal D'$ 上多次训练后，即可得到一个与受害者模型 $f(\cdot)$ 功能和性质类似的替代模型 $f'(\cdot)$ ，完成模型窃取攻击。

<img src="./imgs/model steal2.png" alt="替代模型窃取攻击示意图" style="height: 400px; max-width: 100%;">



为了实现这样的模型窃取方法，我们首先需要加载受害者模型。

（在实际中，模型窃取攻击者通常使用受害者API进行查询访问）

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

import os
import random
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

import datasets

import cifar_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model_path = "./models/"
model_list = [
    "cifar10_resnet18.pth",
    "cifar10_model.pth"
]

# 加载模型
def load_model(model_path, num_classes, device):
    # 加载模型结构
    model = cifar_model.ResNet18(num_classes=num_classes)

    # 加载模型权重
    if device == 'cpu':
        state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    else:
        state_dict = torch.load(model_path)
    
    # 加载权重到模型
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()  # 设置为评估模式

    print("加载模型: ", model_path)
    return model

In [ ]:
victim_model = load_model(model_path + model_list[0], num_classes=10, device=device)

对加载的受害者模型进行测试

In [ ]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# 加载CIFAR-10训练数据集
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_test)
trainloader = DataLoader(trainset, batch_size=100, shuffle=True, num_workers=4, pin_memory=True)

# 加载CIFAR-10测试数据集
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=100, shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
# 测试模型
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
def test_model(model, dataloader, perturbation=None):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            if perturbation: outputs = perturbation(outputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    acc = 100.*correct/total
    return acc

In [ ]:
print("Model Test Accuracy: ", test_model(victim_model, testloader), '%')

接下来，我们作为攻击者，对受害者模型进行轮询访问，以获取其查询-预测对:

现在有一个问题：攻击者如何获取用于查询的数据集？

虽然在实际中，攻击者并不知道模型的任何先验知识，但是知道其功能是什么。举一个简单的例子：受害者模型是一个对猫狗的二分类问题，攻击者即可收集猫和狗的图像作为对模型的查询数据集。

在本次实验中，受害者模型是一个10-分类问题，简单起见，我们直接对 cifar-10 公开数据集进行随机采样，以作为对受害者模型的查询数据集。

In [ ]:
def build_query_set(dataset_name="cifar10", data_size=10000, use_public_data=True):
    """构建用于查询受害者模型的数据集"""
    if use_public_data:

        if dataset_name == "cifar10":
            # TODO: 取 trainset 中一部分(data_size)作为查询数据集
    
    else:
        # 生成随机噪声数据
        return torch.randn(data_size, 3, 32, 32) * 0.5 + 0.5

于是，我们便可以使用查询数据集对受害者模型进行轮询：

In [ ]:
def query_victim_model(victim_model, query_images, batch_size=64, device='cuda', output_perturbation=None):
    """向受害者模型发送查询并收集预测结果"""
    victim_model.to(device)
    victim_model.eval()
    
    all_predictions = []
    dataloader = DataLoader(query_images, batch_size=batch_size, shuffle=False)
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Querying victim model"):

            # TODO：完成对模型的查询

            # 1. 对一个 batch 进行预测， 获取logits输出
            
            # 2. 若 output_perturbation 输出扰动函数不为None，则对logits层输出进行扰动，获取扰动的logits输出

            # 3. 返回受害者模型预测的概率分布（非logits分数）并加入到all_prediction中
    
    # 合并所有批次的预测概率分布
    all_predictions = torch.cat(all_predictions)
    return all_predictions

将获取到的查询-预测对构建成为替代数据集：

In [ ]:
class SubstituteDataset(Dataset):
    def __init__(self, queries, predictions, transform=None):
        self.queries = queries
        self.predictions = predictions
        self.transform = transform
        
    def __len__(self):
        return len(self.queries)
    
    def __getitem__(self, idx):
        img, prediction = self.queries[idx], self.predictions[idx]
        
        if isinstance(img, np.ndarray):
            img = torch.tensor(img)
            
        if self.transform:
            img = self.transform(img)
            
        return img, prediction

def create_substitute_dataset(victim_model, query_images, save_path="substitute_dataset.pt", augment=False, output_perturbation=None):
    """构建并保存替代数据集"""
    print(output_perturbation)
    # 查询受害者模型
    predictions = query_victim_model(victim_model, query_images, output_perturbation=output_perturbation)

    # # 定义数据增强
    # train_transform = transforms.Compose([
    #     transforms.ToPILImage(),
    #     transforms.RandomCrop(32, padding=4),
    #     transforms.RandomHorizontalFlip(),
    #     transforms.ToTensor(),
    #     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    # ])
    
    # 创建替代数据集
    substitute_dataset = SubstituteDataset(query_images, predictions, train_transform if augment else None)
    
    # 保存数据集
    torch.save({
        'queries': query_images,
        'predictions': predictions
    }, save_path)
    
    print(f"替代数据集已保存至: {save_path}")
    print(f"数据集大小: {len(substitute_dataset)} 样本")
    return substitute_dataset

In [ ]:
# 构建查询集（使用CIFAR-10测试集的3000个样本）
query_images = build_query_set(data_size=3000)

# 创建替代数据集
substitute_dataset = create_substitute_dataset(
    victim_model=victim_model,
    query_images=query_images,
    save_path="substitute_dataset_cifar10.pt"
)

我们来看一下替代数据集是什么样的：

In [ ]:
CIFAR10_CLASS_NAMES = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

def visualize_substitute_dataset(dataset, num_samples=5, class_names=CIFAR10_CLASS_NAMES):
    """可视化替代数据集中的样本"""
    indices = np.random.choice(len(dataset), num_samples, replace=False)
    
    plt.figure(figsize=(15, 3))
    for i, idx in enumerate(indices):
        # image, label = dataset[idx]
        image, prob = dataset[idx]
        label = torch.argmax(prob).item()
        # 反归一化以便显示
        image = image.permute(1, 2, 0).numpy()
        image = image * np.array([0.2023, 0.1994, 0.2010]) + np.array([0.4914, 0.4822, 0.4465])
        image = np.clip(image, 0, 1)
        
        # 获取类别名称
        class_name = class_names[label] if label < len(class_names) else f"Unknown ({label})"
        
        plt.subplot(1, num_samples, i+1)
        plt.imshow(image)
        plt.title(f"Predicted: {class_name}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 可视化样本
visualize_substitute_dataset(substitute_dataset)  

获取替代数据集后，我们便可以对自己的模型进行训练了。

作为攻击者，我们并不知道模型内部的架构，因此，我们需要自定义自己的模型架构。

In [ ]:
class SubstituteModel(nn.Module):
    def __init__(self, num_classes=10):
        super(SubstituteModel, self).__init__()
        self.features = nn.Sequential(
            # 第一层卷积
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),
            
            # 第二层卷积
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),
            
            # 第三层卷积
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),
        )
        
        # 全连接分类器
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
substitute_model = SubstituteModel(num_classes=10).to(device)

接下来，我们使用窃取构建的数据集对模型进行训练：

In [ ]:
# 定义两个概率分布之间的交叉熵损失函数
def loss_ce(p, q):
    # 这里的 p 和 q 是经过 softmax 后的概率分布
    # 计算交叉熵损失
    return -(q * p.log()).sum(dim=1).mean()  

def train_substitute_model(substitute_model, substitute_dataset, epochs=50, batch_size=128, lr=0.001, device='cuda', model_name='substitute_model.pth'):
    """在替代数据集上训练替代模型"""
    # 创建数据加载器
    train_loader = DataLoader(substitute_dataset, batch_size=batch_size, shuffle=True)
    
    # 定义损失函数和优化器
    # criterion = nn.CrossEntropyLoss()
    criterion = loss_ce
    optimizer = optim.Adam(substitute_model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    
    # 训练循环
    substitute_model.to(device)
    best_acc = 0.0
    
    for epoch in range(epochs):
        substitute_model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, targets in train_loader:
            # 这里的 targets 是受害者模型关于这个 input 预测的概率分布
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = substitute_model(inputs)
            # loss = criterion(outputs, targets)
            # 根据受害者模型预测的概率分布计算损失
            loss = criterion(F.softmax(outputs, dim=1), targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            # correct += predicted.eq(targets).sum().item()
            correct += (predicted == targets.argmax(dim=1)).sum().item()
        
        # 计算准确率和平均损失
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100. * correct / total
        
        print(f'Epoch {epoch+1}/{epochs} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}%')
        
        # 学习率调度
        scheduler.step(epoch_loss)
        
        # 保存最佳模型
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            torch.save(substitute_model.state_dict(), model_name)
            print(f'Saved best model with accuracy: {best_acc:.2f}%')
    
    print(f'Finished training. Best accuracy: {best_acc:.2f}%')
    return substitute_model

In [ ]:
# 在替代数据集上训练替代模型
print("开始训练替代模型...")
trained_substitute_model = train_substitute_model(
    substitute_model=substitute_model,
    substitute_dataset=substitute_dataset,
    epochs=20,
    device=device
)

接下来，我们对窃取数据集上训练的模型进行评估：

In [ ]:
print("Model Test Accuracy: ", test_model(substitute_model, testloader), '%')

可以发现，窃取模型攻击利用对受害者模型的少量查询，获取了表现“还可以”的模型，这样，攻击者就可以用极低的时间和训练成本来获取一个模型。

# 2. 模型窃取防御

模型窃取防御的目的是让攻击者无法通过简单的查询就能获取模型输出分布/模型参数。

一般来说，被窃取模型准确率越高，它们输出的信息分布就会过于精确，导致攻击者很容易获取到高质量的 输入-输出 对作为高质量数据集，进而通过高质量的数据集来训练出替代模型。

模型窃取防御可以从这个方向出发，通过**模糊化技术**，例如模糊决策边界、模糊输入概率等等方式，对窃取攻击进行防御。

## 2.1 信息模糊
对受害者模型的**输出概率向量**进行模糊化操作，使得输出的向量包含更少、更粗糙的信息，从而增大窃取攻击的学习难度

Shokri 等人在 2017 年提出了四种针对模型窃取的防御方法：
1. **输出向量限制为前 k 个类**：极端情况下，模型只返回最大可能的类别标签
2. **输出向量四舍五入**：将输出向量中的logits值保留 $d$ 位小数，随着 $d$ 值减小，模型泄露的信息也就越小
3. **增加输出向量的信息熵**：向 softmax 层增加调节参数 $\tau$ ，计算 $\text{softmax}(z)=\frac {e^{z_i/\tau}}{\sum\limits_j e^{z_j/\tau}}$ 。随着 $\tau$ 的增大，输出向量变得更均匀，输出向量的熵增大，泄露的信息也就越少。
4. **正则化**：避免模型因为过拟合学习到敏感的决策边界从而泄露敏感信息


在本次实验中，我们尝试前三种方法，即**限制为前 k 个类、输出向量四舍五入到 d 位、增加输出信息熵**来实验其防御效果。下面的代码定义了一层模型的“防御层”，并将其拼接至原模型的最后一层之后形成一个新模型，用于模糊模型的输出信息。而该操作不会影响模型的性能，因为模型预测的最大标签不会改变。

In [ ]:
class defense_layer(nn.Module):
    """
    自定义一个简单的防御层，用于拼接到原受害者模型最后一层之后
    pytorch 通过继承 nn.Module 来定义一个模型层，具体的细节可参考官方文档 `https://docs.pytorch.org/docs/stable/notes/modules.html`
    该层只保留输出 x 中的前 k 个值，将每个值四舍五入到第 d 位，并且根据系数 tau 进行缩放（整体除以或乘 tau）
    """
    def __init__(self, k=5, d=2, tau=None):
        """
        __init__ 函数在定义该层时会被调用，用于初始化一些参数
        用来初始化这个模型层的参数
        """
        super().__init__()
        self.k = k
        self.d = d
        self.tau = tau

    def forward(self, x):
        """
        forward 函数在模型计算（如model(input)）的时候会被调用
        x 代表传入这一层的输入，该函数的 return 返回值代表 x 经过这一层之后的结果
        在自定义模型层时需要定义
        """

        # TODO: 完成防御层的前向传播过程

        # 1. 四舍五入到第 d 位
        
        # 2. 前 k 大位置的值保留，其余位置设为负无穷
        
        # 3. 根据 tau 进行缩放
        
        return new_x

class DefenseModel(nn.Module):
    """
    定义一个防御模型，将受害者模型和防御层拼接在一起
    该模型的 forward 函数会先调用受害者模型的 forward 函数，然后将输出传入防御层，再调用防御层的 forward 函数
    """
    def __init__(self, model, k=5, d=2, tau=None):
        super().__init__()
        self.model = model
        self.defense_layer = defense_layer(k=k, d=d, tau=tau)

    def forward(self, x):
        x = self.model(x)
        x = self.defense_layer(x)
        return x

defense_model = DefenseModel(victim_model, k=3, d=1, tau=20).to(device)
defense_model.eval()

In [ ]:
print("Model Test Accuracy: ", test_model(defense_model, testloader), '%')

可以看到，新模型相比原受害者模型的性能几乎一致。

In [ ]:
# 构建查询集（使用CIFAR-10测试集的3000个样本）
query_images = build_query_set(data_size=3000)

# 创建替代数据集
substitute_dataset_defense = create_substitute_dataset(
    victim_model=defense_model,
    query_images=query_images,
    save_path="substitute_dataset_defense_cifar10.pt"
)

In [ ]:
substitute_model_defense = SubstituteModel(num_classes=10).to(device)

In [ ]:
# 在模糊后的替代数据集上训练替代模型
print("开始训练替代模型...")
trained_substitute_model_defense = train_substitute_model(
    substitute_model=substitute_model_defense,
    substitute_dataset=substitute_dataset_defense,
    epochs=20,
    device=device
)

此时替代模型的训练准确率大幅下降，可见防御层干扰了替代模型的训练过程。我们来看看此时替代模型在测试集上的训练效果。

In [ ]:
print("Model Test Accuracy: ", test_model(substitute_model_defense, testloader), '%')

可以发现，新训练的模型在测试集上的准确率有接近 $10\%$ 的下降，可见防御层起到了一定的效果

## 2.2 模型置信度模糊
除了取 top-k 的预测类别进行输出，模糊化输出外，防御者还可以通过对模型的logits层进行扰动操作，让模型的输出分布受到改变，进而让攻击者获取到低质量的数据集。

这里提供一种方法：根据带温度的softmax来计算每个类别的概率分布，对于某些类别，它的最高概率若不达到阈值，则看作低置信度样本，对其扰动。

扰动策略：
1. 选择一个与原预测不同的随机类别。
2. 将原预测类别的logits缩放（如0.5%）。
3. 将随机选择的类别logits放大10倍。

In [ ]:
import torch
import torch.nn.functional as F

def perturb_prediction(outputs, confidence_threshold=0.60, temperature=1.75):
    """
    基于置信度对模型预测结果进行扰动, 首先是用softmax和温度来计算预测概率分布
    , 找出置信度(最大预测概率)低于阈值confidence_threshold的样本, 这些样本属于不易学习的样本.
    对这些硬样本进行输出logits层的扰动, 模型原预测类别的logits值降低(如0.5%), 提高随机类别的logits
    值(如10倍)。
    
    参数:
        outputs: 模型原始输出logits (batch_size, num_classes)
        confidence_threshold: 置信度阈值，低于此值的预测将被扰动
        temperature: softmax温度参数，控制概率分布的平滑度
        device: 计算设备
        
    返回:
        扰动后的输出logits
    """

    # TODO: 完成对模型预测logits层输出的扰动

    # 1. 计算softmax概率和最大置信度（最大预测概率）
    
    
    # 2. 识别需要扰动的低置信度样本
    
    
    # print(f"低置信度样本比例 ({confidence_threshold}): {(mask.sum()/len(mask)):.2%}")
    
    # 3. 如果没有低置信度样本，直接返回原始输出
    
    
    # 4. 对低置信度样本进行扰动: 
    #      1. 对每个样本生成随机标签(不等于原预测)
    #      2. 把原预测logits分数降低，如 0.5%
    #      3. 把随机标签logits分数提高，如 10 倍
    
    
    return new_outputs

在这个输出扰动的策略下，我们对受害者模型进行输入-输出数据集的窃取：

In [ ]:
# 构建查询集（使用CIFAR-10测试集的3000个样本）
query_images = build_query_set(data_size=3000)

# 创建替代数据集
substitute_dataset = create_substitute_dataset(
    victim_model=victim_model,
    query_images=query_images,
    save_path="substitute_dataset_cifar10.pt",
    # output_perturbation = perturb_prediction
    output_perturbation=perturb_prediction
)

我们在这个受到输出扰动的数据集上，对替代模型进行训练：

In [ ]:
substitute_model_output_perturbation_defense = SubstituteModel(num_classes=10)

In [ ]:
# 在替代数据集上训练替代模型
print("开始训练替代模型...")
trained_substitute_model = train_substitute_model(
    substitute_model=substitute_model_output_perturbation_defense,
    substitute_dataset=substitute_dataset,
    epochs=20,
    device=device
)

测试受害者模型和替代模型的性能：

In [ ]:
print("Model Test Accuracy: ", test_model(victim_model, testloader, perturbation=perturb_prediction), '%')

In [ ]:
print("Model Test Accuracy: ", test_model(substitute_model_output_perturbation_defense, testloader), '%')

可以发现受害者模型的准确率会因为logits层输出扰动而受到一定影响，但是攻击者所训练出的替代模型也会受到较大影响，较好地实现了在受害者模型性能影响不高的情况下，对模型安全性的保障。